In [ ]:
# install beautifulsoup and parser

#!conda install beautifulsoup4

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    cryptography-2.5           |   py36h1ba5d50_0         643 KB
    ca-certificates-2019.1.23  |                0         126 KB
    soupsieve-1.7.1            |           py36_0          50 KB
    conda-4.6.2                |           py36_0         1.6 MB
    grpcio-1.16.1              |   py36hf8bcb03_1         1.1 MB
    libarchive-3.3.3           |       h5d8350f_5         1.5 MB
    certifi-2018.11.29         |           py36_0         146 KB
    beautifulsoup4-4.7.1       |           py36_1         143 KB
    python-3.6.8               |       h0371630_0        34.4 MB
    ------------------------------------------------------------
                                           Total:        39.7 

In [ ]:
#!conda install lxml

1. Import libraries
2. Specify the source file

In [1]:
from bs4 import BeautifulSoup
import requests
import lxml

In [39]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

#print(soup.prettify())

### Collecting information from webpage and cleaning data
1. Parse the html file and read needed information into a dataframe
2. Drop the lines where Borough = 'Not assigned'
3. Combine line with same postcode but different neighbourhoods, separated by comma
4. Fill 'Neighbourhood' values with 'Borough' value when neighbourhood is 'Not assigned'
5. Print the number of rows of the cleaned dataframe

In [119]:
import pandas as pd
import numpy as np

table = soup.find('table',class_='wikitable sortable')

Toronto = pd.DataFrame(columns = ['Postcode','Borough','Neighbourhood'])

for obs in table.find_all('tr'):
    line = obs.text.split('\n')
    postcode = line[1]
    borough = line[2]
    neighbourhood = line[3]
    Toronto = Toronto.append({'Postcode':postcode,'Borough':borough,'Neighbourhood':neighbourhood}, ignore_index=True)
    
Toronto = Toronto.drop(0)
Toronto = Toronto[Toronto['Borough']!='Not assigned']


In [296]:
Toronto_bin=Toronto.groupby(['Postcode','Borough'])['Neighbourhood'].apply(list).to_frame(name="Neighbourhood").reset_index()

Toronto_bin.loc[Toronto_bin.Neighbourhood.apply(lambda x:  "Not assigned" in x ),'Neighbourhood'] = Toronto_bin.Borough


#Toronto_bin[Toronto_bin.Postcode=='M7A']

#Toronto_bin.head(10)


,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [300]:
print ('the number of rows for cleaned Toronto data is',Toronto_bin.shape[0])

the number of rows for cleaned Toronto data is 103


## Add coordinates to previous prepared data Toronto_bin
1. Get the coordinate file from the provided link
2. Merge the coordinate file with neighbourhood file 'Toronto_bin'

In [308]:
!wget -q -O 'Geospatial_Coordinates.csv' https://cocl.us/Geospatial_data

Toronto_coordinates = pd.read_csv('Geospatial_Coordinates.csv')


In [312]:
t_merge = pd.merge(Toronto_bin, Toronto_coordinates, left_on = 'Postcode',right_on='Postal Code')

t_merge=t_merge.drop('Postal Code',axis=1)
t_merge.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"[Rouge, Malvern]",43.806686,-79.194353
1,M1C,Scarborough,"[Highland Creek, Rouge Hill, Port Union]",43.784535,-79.160497
2,M1E,Scarborough,"[Guildwood, Morningside, West Hill]",43.763573,-79.188711
3,M1G,Scarborough,[Woburn],43.770992,-79.216917
4,M1H,Scarborough,[Cedarbrae],43.773136,-79.239476
5,M1J,Scarborough,[Scarborough Village],43.744734,-79.239476
6,M1K,Scarborough,"[East Birchmount Park, Ionview, Kennedy Park]",43.727929,-79.262029
7,M1L,Scarborough,"[Clairlea, Golden Mile, Oakridge]",43.711112,-79.284577
8,M1M,Scarborough,"[Cliffcrest, Cliffside, Scarborough Village West]",43.716316,-79.239476
9,M1N,Scarborough,"[Birch Cliff, Cliffside West]",43.692657,-79.264848


## Start Exploring Toronto Neighbourhood
1. Import libraries needed for the analysis
2. Only consider boroughs that contain word "Toronto"
3. Generate maps to visualize the chosen neighbourhood

In [348]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import json

from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [314]:
Toronto_data = t_merge[t_merge['Borough'].apply(lambda x: 'Toronto' in x)].reset_index(drop=True)
Toronto_data.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,[The Beaches],43.676357,-79.293031
1,M4K,East Toronto,"[The Danforth West, Riverdale]",43.679557,-79.352188
2,M4L,East Toronto,"[The Beaches West, India Bazaar]",43.668999,-79.315572
3,M4M,East Toronto,[Studio District],43.659526,-79.340923
4,M4N,Central Toronto,[Lawrence Park],43.728020,-79.388790


In [317]:
address = 'Toronto, Ca'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [335]:
# Create map of Toronto
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighbourhood']):
    label = ','.join(label)
   
    label = folium.Popup(label, parse_html=True)
    
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [336]:
CLIENT_ID = 'XYK2G5XLJP5KUOWHNW3JOGNPEWAN0JJPMIGWI4FR4J4PBOED' # your Foursquare ID
CLIENT_SECRET = 'N2JRIJN1FQX2FUKND1NUTNNQSBBUYKUINHKM0S4ZKV5X2R1Z' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [341]:
neighborhood_latitude = Toronto_data.loc[1, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = Toronto_data.loc[1, 'Longitude'] # neighborhood longitude value

neighborhood_name =','.join(Toronto_data.loc[1, 'Neighbourhood']) # neighborhood name

print('Latitude and longitude values of neighbourhoods {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of neighbourhoods The Danforth West,Riverdale are 43.6795571, -79.352188.


In [342]:

LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XYK2G5XLJP5KUOWHNW3JOGNPEWAN0JJPMIGWI4FR4J4PBOED&client_secret=N2JRIJN1FQX2FUKND1NUTNNQSBBUYKUINHKM0S4ZKV5X2R1Z&v=20180605&ll=43.6795571,-79.352188&radius=500&limit=100'

In [345]:
results = requests.get(url).json()
#results

In [346]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Clean the json and structure it into a *pandas* dataframe.

In [349]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Pantheon,Greek Restaurant,43.677621,-79.351434
1,Dolce Gelato,Ice Cream Shop,43.677773,-79.351187
2,MenEssentials,Cosmetics Shop,43.677820,-79.351265
3,Messini Authentic Gyros,Greek Restaurant,43.677827,-79.350569
4,Mezes,Greek Restaurant,43.677962,-79.350196


## Use the function provided in lab session to explore all the neighborhoods in Toronto

In [351]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        try:    
        # make the GET request
          results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
          venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
            
        except:    
            print('unable to fetch data')
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
       
    return(nearby_venues)

In [364]:

Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )



['The Beaches']
['The Danforth West', 'Riverdale']
['The Beaches West', 'India Bazaar']
['Studio District']
['Lawrence Park']
['Davisville North']
['North Toronto West']
['Davisville']
['Moore Park', 'Summerhill East']
['Deer Park', 'Forest Hill SE', 'Rathnelly', 'South Hill', 'Summerhill West']
['Rosedale']
['Cabbagetown', 'St. James Town']
['Church and Wellesley']
['Harbourfront', 'Regent Park']
['Ryerson', 'Garden District']
['St. James Town']
['Berczy Park']
['Central Bay Street']
['Adelaide', 'King', 'Richmond']
['Harbourfront East', 'Toronto Islands', 'Union Station']
['Design Exchange', 'Toronto Dominion Centre']
['Commerce Court', 'Victoria Hotel']
['Roselawn']
['Forest Hill North', 'Forest Hill West']
['The Annex', 'North Midtown', 'Yorkville']
['Harbord', 'University of Toronto']
['Chinatown', 'Grange Park', 'Kensington Market']
['CN Tower', 'Bathurst Quay', 'Island airport', 'Harbourfront West', 'King and Spadina', 'Railway Lands', 'South Niagara']
['Stn A PO Boxes 25 The Es

### check how many venues were fround in each neighborhood

In [365]:
Toronto_venues.Neighborhood = Toronto_venues.Neighborhood.apply(lambda x: ''.join(x))

#Toronto_venues.head()

Toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
AdelaideKingRichmond,100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
BrocktonExhibition PlaceParkdale Village,18,18,18,18,18,18
Business Reply Mail Processing Centre 969 Eastern,18,18,18,18,18,18
CN TowerBathurst QuayIsland airportHarbourfront WestKing and SpadinaRailway LandsSouth Niagara,14,14,14,14,14,14
CabbagetownSt. James Town,46,46,46,46,46,46
Central Bay Street,82,82,82,82,82,82
ChinatownGrange ParkKensington Market,100,100,100,100,100,100
Christie,16,16,16,16,16,16


### find out the number of unique categories for returned neighorhoods

In [366]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 237 uniques categories.


## Analyzing each neighborhood

In [368]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [372]:
Toronto_onehot.shape

(1697, 237)

#### Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [388]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,AdelaideKingRichmond,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.01,0.0,0.0,0.01,0.0,0.01
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
2,BrocktonExhibition PlaceParkdale Village,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.055556,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00
4,CN TowerBathurst QuayIsland airportHarbourfron...,0.000000,0.0,0.0,0.071429,0.071429,0.071429,0.142857,0.142857,0.142857,...,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.0,0.00


#### Print each neighborhood along with the top 3 most common venues

In [374]:
num_top_venues = 3

for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----AdelaideKingRichmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04


----Berczy Park----
          venue  freq
0   Coffee Shop  0.07
1  Cocktail Bar  0.05
2    Restaurant  0.05


----BrocktonExhibition PlaceParkdale Village----
            venue  freq
0            Café  0.11
1  Breakfast Spot  0.11
2     Coffee Shop  0.11


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.06
2                Park  0.06


----CN TowerBathurst QuayIsland airportHarbourfront WestKing and SpadinaRailway LandsSouth Niagara----
              venue  freq
0   Airport Service  0.14
1  Airport Terminal  0.14
2    Airport Lounge  0.14


----CabbagetownSt. James Town----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.07
2         Park  0.07


----Central Bay Street----
                venue  freq
0         Coffee Shop  0.16
1           

#### Put that into a *pandas* dataframe

1. Use the function in lab session that sort venues in descending order
2. Create new dataframe and display top 10 venues from each neighborhood

In [376]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [393]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,AdelaideKingRichmond,Coffee Shop,Café,American Restaurant,Thai Restaurant,Steakhouse,Asian Restaurant,Bakery,Bar,Gym,Clothing Store
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Steakhouse,Cheese Shop,Café,Pub,Farmers Market,Beer Bar,Italian Restaurant
2,BrocktonExhibition PlaceParkdale Village,Coffee Shop,Breakfast Spot,Café,Performing Arts Venue,Convenience Store,Burrito Place,Bar,Stadium,Caribbean Restaurant,Italian Restaurant
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Yoga Studio,Auto Workshop,Pizza Place,Butcher,Recording Studio,Restaurant,Burrito Place,Brewery,Skate Park
4,CN TowerBathurst QuayIsland airportHarbourfron...,Airport Lounge,Airport Terminal,Airport Service,Boutique,Plane,Boat or Ferry,Sculpture Garden,Airport Gate,Airport Food Court,Airport


###  Cluster Neighborhoods

1. use K-means with 5 clusters
2. visualize the clusters

In [394]:
# set number of clusters
kclusters = 5

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [395]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data.rename(columns={'Neighbourhood':'Neighborhood'})
Toronto_merged.Neighborhood = Toronto_merged.Neighborhood.apply(lambda x: ''.join(x))

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head(3)

,Postcode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Coffee Shop,Pub,Asian Restaurant,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store
1,M4K,East Toronto,The Danforth WestRiverdale,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Grocery Store,Brewery,Bubble Tea Shop,Restaurant,Pub
2,M4L,East Toronto,The Beaches WestIndia Bazaar,43.668999,-79.315572,0,Sandwich Place,Pet Store,Park,Brewery,Burger Joint,Burrito Place,Pub,Pizza Place,Movie Theater,Sushi Restaurant


In [396]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [398]:
Toronto_merged['Cluster Labels'].unique()

array([0, 4, 1, 2, 3])

In [399]:

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

##  Examine Clusters

In [406]:
for label in [0,1,2,3,4]:
    
    print('current label is',label)
    
    tt =Toronto_merged.loc[Toronto_merged['Cluster Labels'] == label, Toronto_merged.columns[[1] + list(range(5, Toronto_merged.shape[1]))]]
    
    print(tt)
    

current label is 0
             Borough  Cluster Labels 1st Most Common Venue  \
0       East Toronto               0           Coffee Shop   
1       East Toronto               0      Greek Restaurant   
2       East Toronto               0        Sandwich Place   
3       East Toronto               0                  Café   
4    Central Toronto               0              Bus Line   
5    Central Toronto               0        Clothing Store   
6    Central Toronto               0           Coffee Shop   
7    Central Toronto               0          Dessert Shop   
9    Central Toronto               0                   Pub   
11  Downtown Toronto               0           Coffee Shop   
12  Downtown Toronto               0   Japanese Restaurant   
13  Downtown Toronto               0           Coffee Shop   
14  Downtown Toronto               0           Coffee Shop   
15  Downtown Toronto               0           Coffee Shop   
16  Downtown Toronto               0           Coff